In [2]:
%pip install --quiet bitsandbytes transformers datasets accelerate wandb

     |████████████████████████████████| 62.5 MB 1.3 MB/s 
     |████████████████████████████████| 5.5 MB 51.0 MB/s 
     |████████████████████████████████| 451 kB 66.4 MB/s 
     |████████████████████████████████| 175 kB 64.4 MB/s 
     |████████████████████████████████| 1.9 MB 57.1 MB/s 
     |████████████████████████████████| 7.6 MB 50.5 MB/s 
     |████████████████████████████████| 182 kB 69.2 MB/s 
     |████████████████████████████████| 212 kB 65.7 MB/s 
     |████████████████████████████████| 115 kB 68.5 MB/s 
     |████████████████████████████████| 127 kB 69.6 MB/s 
     |████████████████████████████████| 168 kB 69.0 MB/s 
     |████████████████████████████████| 182 kB 69.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 69.8 MB/s 
     |████████████████████████████████| 166 kB 71.4 MB/s 
     |████████████████████████████████| 162 kB 72.0 MB/s 
     |████████████████████████████████| 162 kB 73.4 MB/s 
     |██████████

In [ ]:
import torch
torch.__version__

'1.12.1+cu113'

In [ ]:
import transformers
transformers.__version__

'4.24.0'

In [3]:
!wget https://raw.githubusercontent.com/AIshutin/opendungeon/master/questdataset.py
!wget https://huggingface.co/datasets/OpenDungeon/chooseyourstory/resolve/main/data_train_mxlen1000.pkl
!wget https://huggingface.co/datasets/OpenDungeon/chooseyourstory/resolve/main/data_test_mxlen1000.pkl
!wget https://huggingface.co/datasets/OpenDungeon/chooseyourstory/resolve/main/data_val_mxlen1000.pkl

--2022-11-18 12:21:34--  https://raw.githubusercontent.com/AIshutin/opendungeon/master/questdataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5751 (5.6K) [text/plain]
Saving to: ‘questdataset.py’

questdataset.py     100%[===================>]   5.62K  --.-KB/s    in 0s      

2022-11-18 12:21:34 (74.8 MB/s) - ‘questdataset.py’ saved [5751/5751]

--2022-11-18 12:21:34--  https://huggingface.co/datasets/OpenDungeon/chooseyourstory/resolve/main/data_train_mxlen1000.pkl
Resolving huggingface.co (huggingface.co)... 34.200.207.32, 34.227.196.80, 2600:1f18:147f:e800:7a81:37eb:4345:da2, ...
Connecting to huggingface.co (huggingface.co)|34.200.207.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.c

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

MODEL_NAME = "EleutherAI/gpt-j-6B" # "EleutherAI/gpt-j-6B" #"facebook/opt-6.7b" # 

model = AutoModelForCausalLM.from_pretrained(
  MODEL_NAME, device_map='auto', load_in_8bit=True,
  low_cpu_mem_usage=True, torch_dtype=torch.float16, offload_state_dict=True,
  cache_dir="mycache")

# I have no idea which of the from_pretrained params are necessary ~.~
# If you are reading this and you not in (Tim,Younes,Felix), we messed up

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

for module in model.modules():
    if isinstance(module, bnb.nn.Linear8bitLt):
        module.state.memory_efficient_backward = True

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
class RequiresGradWrapper(nn.Sequential):
    def forward(self, *args, **kwargs):
        return super().forward(*args,**kwargs).requires_grad_(True)
        
model.transformer.wte = RequiresGradWrapper(model.transformer.wte)
model.model.decoder.project_in = lambda x: x.requires_grad_(True)


# cast model outputs to float32 to unfuck the top-k sampler
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

Downloading:   0%|          | 0.00/651 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/41.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

AttributeError: ignored

In [7]:
model.model

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 4096, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
    (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0): OPTDecoderLayer(
        (self_attn): OPTAttention(
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (out_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear8bitLt(in_features=4096, out_features=16384, bias=True)
        (fc2): Linear8bitLt(in_features=16384, out_features=4096, bias=True)
        (final_layer_norm): LayerNorm((4096,), eps=1e-05, element

In [ ]:
batch = tokenizer("Mark Zuckerberg is", return_tensors='pt')
# note to self: find a less controversial example

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, min_length=30, max_length=30, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0].numpy()))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1449: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/bitsandbytes/autograd/_functions.py:231: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




 </s>Mark Zuckerberg is one of the greatest minds in the history of humanity, and if you disagree that's fine.
The man has made Facebook.


### Apply LoRA

In [ ]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter"""
    def __init__(self, module: nn.Module, rank: int):
        super().__init__()
        self.module = module
        self.adapter = nn.Sequential(nn.Linear(module.in_features, rank, bias=False),
                                     nn.Linear(rank, module.out_features, bias=False))
        small_std = (2. / (5 * min(module.in_features, module.out_features))) ** 0.5
        nn.init.normal_(self.adapter[0].weight, std=small_std)
        nn.init.zeros_(self.adapter[1].weight)
        self.adapter.to(module.weight.device)

    def forward(self, input, *args, **kwargs):
        return self.module(input, *args, **kwargs) + self.adapter(input)


In [ ]:
for name, module in model.named_modules():
  if 'GPTJAttention' in repr(type(module)):
    module.q_proj = LoRALayer(module.q_proj, rank=16)
    module.k_proj = LoRALayer(module.k_proj, rank=16)
    module.v_proj = LoRALayer(module.v_proj, rank=16)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96

In [ ]:
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast():
  out = model.forward(**batch)
  out.logits.norm().backward()

for module in model.modules():
  if isinstance(module, LoRALayer):
    assert module.adapter[1].weight.grad is not None
    assert module.adapter[1].weight.grad.norm().item() > 0

model.zero_grad(set_to_none=True)

In [ ]:
import transformers


class MyModelWrapperThatSaves(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, past_key_values=None, attention_mask=None, *args, **kwargs):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, past_key_values=past_key_values, *args, **kwargs)

    def state_dict(self):
        print('CUSTOM STATE DICT USED')
        adapters = {}
        for name, module in self.model.named_modules():
          if isinstance(module, LoRALayer):
            for key, w in module.adapter.state_dict().items():
              adapters[name + '.adapter.' + key] = w
        return adapters

    def load_state_dict(self, state_dict_adapters_only):
        fixed_layers = {}
        for el in state_dict_adapters_only:
          assert(el in self.model.state_dict())
        self.model.load_state_dict(state_dict_adapters_only, strict=False)
        return self

model_wrapped_for_trainer = MyModelWrapperThatSaves(model)
state_dict_adapters_only = model_wrapped_for_trainer.state_dict()
model_wrapped_for_trainer.load_state_dict(state_dict_adapters_only)
assert not isinstance(model_wrapped_for_trainer, transformers.PreTrainedModel)
assert isinstance(model_wrapped_for_trainer.model, transformers.PreTrainedModel)

SyntaxError: ignored

### Training

QuestDataset

In [ ]:
%load_ext autoreload
%autoreload 2
import questdataset
import transformers
import random
import pickle

train_dataset = questdataset.PresampledDataset(pickle.load(open('data_train_mxlen1000.pkl', 'rb')), tokenizer)
val_dataset = questdataset.PresampledDataset(pickle.load(open('data_val_mxlen1000.pkl', 'rb')), tokenizer)

In [ ]:
len(val_dataset)

In [ ]:
def compute_metrics(eval_preds):
  logits, labels, inputs = eval_preds
  top1 = 0
  top10 = 0
  total = 0
  logits = torch.tensor(logits)

  DM_id = tokenizer.encode("DM")[0]
  P_id = tokenizer.encode("Player")[0]

  for si in range(inputs.shape[0]):
    c = (inputs[si] != 1).sum()
    middle = c // 2 + 1
    mx = c - 1
    state = 1

    for tj in range(0, mx):
      if inputs[si, tj] == DM_id:
        state = 0
      elif inputs[si, tj] == P_id:
        state = 1
      if tj < middle:
        continue
      if state != 0:
        continue
      total += 1
      top1 += logits[si, tj].argmax() == inputs[si, tj]
      top10 += inputs[si, tj] in logits[si, tj].topk(10).indices #.topk(10)
  return {"acc-top1": top1 / total, "acc-top10": top10 / total}

In [ ]:
import transformers

STEPS = 1000 # if bs=16 this is 2 times per epoch.

trainer = transformers.Trainer(
  model=model_wrapped_for_trainer, train_dataset=train_dataset, eval_dataset=val_dataset,
  args=transformers.TrainingArguments(
    per_device_train_batch_size=1, gradient_accumulation_steps=16, #gas should be 16
    warmup_steps=250000 / 500, num_train_epochs=50, learning_rate=2e-4, fp16=True, # maybe 10k * 2 / 32k ~ 1 epoch will be enough; 2e-4 (default and the only) lora lr; warmup steps = 500. 
    logging_steps=STEPS, 
    output_dir='outputs', report_to="wandb",
    save_strategy="steps", save_steps=STEPS,
    evaluation_strategy="steps", eval_steps=STEPS,
    include_inputs_for_metrics=True
  ),
  data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False), 
  compute_metrics=compute_metrics,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
wandb.finish()

### Optimize for inference (forever!)

This converts training-optimized 8bit weights into inference-optimized layout. It should result in significantly faster inference in the same memory foorptint. 
However, if you do this, you can no longer run training --
 there is no way to un-convert after the first optimized forward pass!

In [ ]:
model.config.use_cache = True

for module in model.modules():
    if isinstance(module, bnb.nn.Linear8bitLt):
        module.state.memory_efficient_backward = False